The purpose of the project is to learn the mapping from polar coordinates to a a discrete 10x10 grid of cells in the plane, using a neural network. 

The supervised dataset is given to you in the form of a generator (to be considered as a black box).

The model must achieve an accuracy of 95%, and it will be evaluated in a way **inversely proportional to the number of its parameters: the smaller, the better.**

**WARNING**: Any solution taking advantage of meta-knowledge about the generator will be automatically rejected.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

Here is the generator. It returns triples of the form ((theta,rho),out) where (theta,rho) are the polar coordinates of a point in the first quadrant of the plane, and out is a 10x10 map with "1" in the cell corresponding to the point position, and "0" everywhere else.

By setting flat=True, the resulting map is flattened into a vector with a single dimension 100. You can use this variant, if you wish. 

In [2]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Let's create an instance of the generator on a grid with dimension 3x4

In [3]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

And now let's see a few samples.

In [4]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 0
y coordinate (col): 1
map:
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Exercise: add noise to the generator, and check the effect on the "ground truth".

# What to deliver

For the purposes of the project you are supposed to work with the **default 10x10 grid, and the default noise=.002**

The generator must be treatead as a black box, do not tweak it, and do not exploit its semantics that is supposed to be unknown. You are allowed to work with the "flat" modality, if you prefer so.

You need to:
1.   define an accuracy function (take inspiration from the code of the previous cell)
2.   define a neural network taking in input theta and rho, and returning out
3. measure the network's accuracy that must be above 95% (accuracy must be evaluated over at least 20000 samples)
4. tune the network trying to decrease as much as possible the numer of parameters, preserving an accuracy above 95%. Only your best network must be delivered.

You must deliver a SINGLE notebook working on colab, containing the code of the network, its summary, the training history, the code for the accurary metric and its evaluation on the network.

**N.B.** The accuracy must be above 95% but apart from that it does not influence the evaluation. You score will only depend on the number of parameters: the lower, the better.

#Good work!





In [106]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [92]:
def get_network(output_shape=(10,10)):
  #theta, ro = theta_ro
  out1, out2 = output_shape
  out_area = out1 * out2

  input1 = Input(shape=(1,))
  input2 = Input(shape=(1,))
  merged = concatenate([input1, input2])

  #inputs = concatenate([theta, ro])
  #x = Input(shape=input_shape)
  d1 = Dense(np.rint(np.sqrt(out_area)), activation='relu')(merged)
  d2 = Dense(out_area, activation='softmax')(d1)
  
  model = Model(inputs=[input1, input2], outputs=d2)
  print(model.summary())
  return model


In [107]:
net = get_network(output_shape=(10,10))
learning_rate = 1e-3
weight_decay = 1e-4

net.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss=CategoricalCrossentropy()
)


Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_28 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 concatenate_11 (Concatenate)   (None, 2)            0           ['input_27[0][0]',               
                                                                  'input_28[0][0]']               
                                                                                                  
 dense_28 (Dense)               (None, 10)           30          ['concatenate_11[0][0]']  

In [94]:
BATCH_SIZE = 128

callback = EarlyStopping(monitor="loss",
    min_delta=0.01,
    patience=10,
    restore_best_weights=True)

# history = net.fit_generator(polar_generator(BATCH_SIZE), steps_per_epoch=1000, epochs=10)
history = net.fit(polar_generator(BATCH_SIZE, flat=True), batch_size = BATCH_SIZE, steps_per_epoch=1000, epochs=1000, callbacks=[callback])

Epoch 1/100
1000/1000 [==============================] - 4s 3ms/step - loss: 4.1234
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 3.2054
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.6377
Epoch 4/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.2915
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 2.0604
Epoch 6/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8840
Epoch 7/100
1000/1000 [==============================] - 2s 2ms/step - loss: 1.7448
Epoch 8/100
1000/1000 [==============================] - 2s 2ms/step - loss: 1.6360
Epoch 9/100
1000/1000 [==============================] - 2s 2ms/step - loss: 1.5485
Epoch 10/100
1000/1000 [==============================] - 2s 2ms/step - loss: 1.4691
Epoch 11/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.4069
Epoch 12/100
1000/1000 [==============================] - 2s 2ms/step - lo

In [79]:
def get_accuracy(net, gen, iteration=1000):
  (theta,rho),maps = next(gen)
  pred = net.predict((theta,rho))

  ok = 0
  for i in range(iteration):
    if np.argmax(maps[i]) == np.argmax(pred[i]):
      ok += 1

  return ok*100/iteration

In [104]:
get_accuracy(net, polar_generator(50000, grid=(10,10), flat=True), iteration=50000)

1563/1563 [==============================] - 2s 1ms/step


89.338